In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [16]:
df=pd.read_csv("/content/drive/My Drive/stock_model_data_v3.0.csv")
df.shape

(1375085, 49)

In [17]:
df["Date"]=pd.to_datetime(df["Date"])
df_2019_2020=df[(df["Date"] >= "2019-01-01") ]
df_2019_2020.shape

(323375, 49)

In [18]:
df_train=df_2019_2020.iloc[0:1,:]
def prepare_train(df):
  
    global df_train
    df_subset=df[df["Date"]<'2020-01-01']
    df_train=df_train.append(df_subset)
        
df_2019_2020.iloc[1:,:].groupby('Symbols').apply(prepare_train) 
df_train.shape

(281199, 49)

In [19]:
symbols=df_train['Symbols']
dates=df_train['Date']
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df_train_scaled_values=scaler.fit_transform(df_train.iloc[:,2:])

df_train_scaled=pd.DataFrame(df_train_scaled_values,columns=df_train.columns[2:])
df_train_scaled['Symbols']=symbols.values
df_train_scaled['Date']=dates.values
df_train_scaled=df_train_scaled[df_train.columns]
df_train_scaled.head()

,Symbols,Date,bin_gain,bin_gain_1%,bin_gain_2%,SMA_PCH5,SMA_PCH15,SMA_PCH50,EMA_PCH5,EMA_PCH15,EMA_PCH50,ROC5,ROC15,ROC50,MOM_PCH5,MOM_PCH15,MOM_PCH50,macdhist,RSI,StochK,StochD,CMO,AroonUp,AroonDown,ChaikinAD,MFI,WillR,ULTOS,MA_BIN5,MA_BIN15,MA_BIN50,EMA_BIN5,EMA_BIN15,EMA_BIN50,ROC_BIN5,ROC_BIN15,ROC_BIN50,MOM_BIN5,MOM_BIN15,MOM_BIN50,macd_BIN,Beta_DOW,Beta_NDQ,Beta_NKK,Beta_NFTY,Beta_EURINR,Beta_GBPINR,Beta_USDINR,Beta_JPYINR
0,20MICRONS.NS,2019-01-01,0.0,0.0,0.0,0.272888,0.153890,0.118366,0.382373,0.204310,0.132544,0.100874,0.093447,0.092397,0.924910,0.960334,0.964154,0.645913,0.578339,1.000000,0.538462,0.578339,0.44,0.00,0.754551,0.559276,0.850746,0.410535,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.659376,0.621633,0.613479,0.440109,0.374381,0.506552,0.615799,0.372038
1,20MICRONS.NS,2019-01-02,0.0,0.0,0.0,0.268255,0.152835,0.117679,0.376302,0.202387,0.131760,0.099852,0.090290,0.094353,0.924120,0.958868,0.964945,0.646127,0.563013,0.924528,0.744074,0.563013,0.40,1.00,0.754507,0.678977,0.582417,0.382327,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.659186,0.622859,0.612647,0.438002,0.372540,0.506837,0.616610,0.376306
2,20MICRONS.NS,2019-01-03,0.0,0.0,0.0,0.261940,0.150536,0.115880,0.369681,0.198916,0.129764,0.098110,0.087824,0.093363,0.922739,0.957652,0.964549,0.646143,0.518346,0.698114,0.874214,0.518346,0.36,0.96,0.754491,0.639004,0.450549,0.413571,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.665346,0.628724,0.615041,0.449382,0.380856,0.513757,0.612551,0.360319
3,20MICRONS.NS,2019-01-04,1.0,0.0,0.0,0.253621,0.146179,0.112324,0.361238,0.193167,0.125977,0.094405,0.085590,0.095790,0.919647,0.956493,0.965505,0.645930,0.441681,0.245283,0.622642,0.441681,0.32,0.92,0.754478,0.617409,0.186813,0.381936,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.643412,0.603874,0.597839,0.439958,0.366041,0.493383,0.607406,0.363016
4,20MICRONS.NS,2019-01-07,0.0,0.0,0.0,0.255762,0.147810,0.113300,0.366295,0.195381,0.127324,0.095386,0.087391,0.096606,0.920487,0.957432,0.965816,0.645856,0.468440,0.396227,0.446541,0.468440,0.28,0.88,0.754478,0.597991,0.274725,0.397314,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.647341,0.611192,0.597703,0.429781,0.367164,0.497982,0.611768,0.356132


In [20]:
X_train=[]
y_train=[]
y_train_names=[]
def prepare_train_arr(df):
    for i in range(15,len(df)):
        X_train.append(df.iloc[i-15:i,4:].values)
        y_train.append(df.iloc[i,4])
        y_train_names.append(df.iloc[i,0])
    
df_train_scaled.groupby(["Symbols"]).apply(prepare_train_arr)
X_train=np.array(X_train)
y_train=np.array(y_train)
X_train.shape

(261634, 15, 45)

In [21]:
df_symbols=df_2019_2020['Symbols']
df_dates=df_2019_2020['Date']
df_scaled_values=scaler.transform(df_2019_2020.iloc[:,2:])
df_scaled=pd.DataFrame(df_scaled_values,columns=df_2019_2020.columns[2:])
df_scaled['Symbols']=df_symbols.values
df_scaled['Date']=df_dates.values
df_scaled=df_scaled[df_2019_2020.columns]
print(df_scaled.shape)
df_scaled.head()

(323375, 49)


,Symbols,Date,bin_gain,bin_gain_1%,bin_gain_2%,SMA_PCH5,SMA_PCH15,SMA_PCH50,EMA_PCH5,EMA_PCH15,EMA_PCH50,ROC5,ROC15,ROC50,MOM_PCH5,MOM_PCH15,MOM_PCH50,macdhist,RSI,StochK,StochD,CMO,AroonUp,AroonDown,ChaikinAD,MFI,WillR,ULTOS,MA_BIN5,MA_BIN15,MA_BIN50,EMA_BIN5,EMA_BIN15,EMA_BIN50,ROC_BIN5,ROC_BIN15,ROC_BIN50,MOM_BIN5,MOM_BIN15,MOM_BIN50,macd_BIN,Beta_DOW,Beta_NDQ,Beta_NKK,Beta_NFTY,Beta_EURINR,Beta_GBPINR,Beta_USDINR,Beta_JPYINR
0,20MICRONS.NS,2019-01-01,0.0,0.0,0.0,0.272888,0.153890,0.118366,0.382373,0.204310,0.132544,0.100874,0.093447,0.092397,0.924910,0.960334,0.964154,0.645913,0.578339,1.000000,0.538462,0.578339,0.44,0.00,0.754551,0.559276,0.850746,0.410535,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.659376,0.621633,0.613479,0.440109,0.374381,0.506552,0.615799,0.372038
1,20MICRONS.NS,2019-01-02,0.0,0.0,0.0,0.268255,0.152835,0.117679,0.376302,0.202387,0.131760,0.099852,0.090290,0.094353,0.924120,0.958868,0.964945,0.646127,0.563013,0.924528,0.744074,0.563013,0.40,1.00,0.754507,0.678977,0.582417,0.382327,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.659186,0.622859,0.612647,0.438002,0.372540,0.506837,0.616610,0.376306
2,20MICRONS.NS,2019-01-03,0.0,0.0,0.0,0.261940,0.150536,0.115880,0.369681,0.198916,0.129764,0.098110,0.087824,0.093363,0.922739,0.957652,0.964549,0.646143,0.518346,0.698114,0.874214,0.518346,0.36,0.96,0.754491,0.639004,0.450549,0.413571,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.665346,0.628724,0.615041,0.449382,0.380856,0.513757,0.612551,0.360319
3,20MICRONS.NS,2019-01-04,1.0,0.0,0.0,0.253621,0.146179,0.112324,0.361238,0.193167,0.125977,0.094405,0.085590,0.095790,0.919647,0.956493,0.965505,0.645930,0.441681,0.245283,0.622642,0.441681,0.32,0.92,0.754478,0.617409,0.186813,0.381936,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.643412,0.603874,0.597839,0.439958,0.366041,0.493383,0.607406,0.363016
4,20MICRONS.NS,2019-01-07,0.0,0.0,0.0,0.255762,0.147810,0.113300,0.366295,0.195381,0.127324,0.095386,0.087391,0.096606,0.920487,0.957432,0.965816,0.645856,0.468440,0.396227,0.446541,0.468440,0.28,0.88,0.754478,0.597991,0.274725,0.397314,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.647341,0.611192,0.597703,0.429781,0.367164,0.497982,0.611768,0.356132


In [22]:
X_test=[]
y_test=[]
y_test_names=[]
def prepare_test(df):
  if(not any(df.Date>'2020-01-01')):
    return
  start_index=np.where(df.Date>='2020-01-01')[0][0]

  for i in range(start_index,len(df)):
    if(i<15):
      continue
    X_test.append(df.iloc[i-15:i,4:].values)
    y_test.append(df.iloc[i,4])
    y_test_names.append(df.iloc[i,0])
 
df_scaled.groupby(["Symbols"]).apply(prepare_test)
X_test=np.array(X_test)
print(X_test.shape)
y_test=np.array(y_test)

(42001, 15, 45)


In [23]:
df_scaled['bin_gain_2%'].value_counts()

0.0    270302
1.0     53073
Name: bin_gain_2%, dtype: int64

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras import optimizers
import keras
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [0]:
model=Sequential()

model.add(LSTM(units=50,input_shape=(X_train.shape[1],45)))
model.add(Dropout(0.2))

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(256,activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(units=1,activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['binary_accuracy'])

In [30]:
model.fit(X_train, y_train, batch_size=500, nb_epoch=100,validation_split=0.33,callbacks=[es,mc])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 175294 samples, validate on 86340 samples
Epoch 1/100
175294/175294 [==============================] - 14s 80us/step - loss: 0.4460 - binary_accuracy: 0.8344 - val_loss: 0.4435 - val_binary_accuracy: 0.8346

Epoch 00001: val_loss improved from inf to 0.44350, saving model to best_model.h5
Epoch 2/100
175294/175294 [==============================] - 12s 69us/step - loss: 0.4394 - binary_accuracy: 0.8353 - val_loss: 0.4401 - val_binary_accuracy: 0.8349

Epoch 00002: val_loss improved from 0.44350 to 0.44008, saving model to best_model.h5
Epoch 3/100
175294/175294 [==============================] - 12s 67us/step - loss: 0.4381 - binary_accuracy: 0.8353 - val_loss: 0.4378 - val_binary_accuracy: 0.8346

Epoch 00003: val_loss improved from 0.44008 to 0.43779, saving model to best_model.h5
Epoch 4/100
175294/175294 [==============================] - 12s 68us/step - loss: 0.4375 - binary_accuracy: 0.8353 - val_loss: 0.4370 - val_binary_accuracy: 0.8346

Epoch 00004: val_loss improved 

In [0]:
from keras.models import load_model
saved_model = load_model('best_model.h5')

In [32]:
from sklearn.metrics import precision_score,accuracy_score,classification_report
y_predicted=saved_model.predict_classes(X_train)
y_predicted=y_predicted.reshape(1,-1)[0]
precision = precision_score(y_train, y_predicted)
accuracy=accuracy_score(y_train,y_predicted)
print("Train Precision is ",precision)
print("Train Accuracy is ",accuracy)

Train Precision is  0.6835585585585585
Train Accuracy is  0.8362521690605961


In [33]:
pd.crosstab(y_train, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0.0,218185,281,218466
1.0,42561,607,43168
All,260746,888,261634


In [34]:
y_predicted=saved_model.predict_classes(X_test)
y_predicted=y_predicted.reshape(1,-1)[0]
precision = precision_score(y_test, y_predicted)
accuracy=accuracy_score(y_test,y_predicted)
print("Test Precision is ",precision)
print("Test Accuracy is ",accuracy)

Test Precision is  0.6634615384615384
Test Accuracy is  0.8140520463798481


In [35]:
pd.crosstab(y_test, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0.0,34122,35,34157
1.0,7775,69,7844
All,41897,104,42001


In [0]:
### Each day of February 

In [0]:
X_testdaily=[]
y_testdaily=[]
ithday_date=[]

dates=pd.date_range(start="2020-02-01",end="2020-03-01",freq="D")

def prepare_test_daily(df,i):
  global X_test_ithday
  global y_test_ithday

  if(not any(df.Date == i)):
    return
  start_index=np.where(df.Date==i)[0][0]

  if(start_index<15):
      return
      
  X_test_ithday.append(df.iloc[start_index-15:start_index,4:].values)
  
  

  y_test_ithday.append(df.iloc[start_index,4])
  

 

for i in dates:
  X_test_ithday=list()
  y_test_ithday=list()
  
  df_scaled.groupby(["Symbols"]).apply(prepare_test_daily, i)

  X_test_ithday=np.array(X_test_ithday)
  y_test_ithday=np.array(y_test_ithday)

  if(X_test_ithday.shape == (0,)):
    continue

  X_testdaily.append(X_test_ithday)
  y_testdaily.append(y_test_ithday)
  ithday_date.append(i)



In [103]:
len(X_testdaily)

13

In [104]:
precisions=[]
accuracies=[]
for i in range(13):
  ithday_predictions=saved_model.predict_classes(X_testdaily[i])
  ithday_predictions=ithday_predictions.reshape(1,-1)[0]
  precision = precision_score(y_testdaily[i],ithday_predictions)
  # print(precision , i)
  accuracy=accuracy_score(y_testdaily[i],ithday_predictions)
  precisions.append(precision)
  accuracies.append(accuracy)
accuracies=np.array(accuracies)
precisions=np.array(precisions)

print("Average Per Day Accuracy for the month of February 2020 - ",accuracies.mean())
print("Standard Deviation of accuracies for the month of February 2020 -",accuracies.std())
print("\nAverage Precision per Day for the month of February 2020 - ",precisions.mean())
print("Standard Deviations of precisions for the month of February 2020 -",precisions.std())


Average Per Day Accuracy for the month of February 2020 -  0.808915654019345
Standard Deviation of accuracies for the month of February 2020 - 0.09336831793637589

Average Precision per Day for the month of February 2020 -  0.814102564102564
Standard Deviations of precisions for the month of February 2020 - 0.2492592576310715


In [0]:
## Each day of January

In [0]:
X_testdaily=[]
y_testdaily=[]
ithday_date=[]

dates=pd.date_range(start="2020-01-01",end="2020-02-01",freq="D")

def prepare_test_daily(df,i):
  global X_test_ithday
  global y_test_ithday

  if(not any(df.Date == i)):
    return
  start_index=np.where(df.Date==i)[0][0]

  if(start_index<15):
      return
      
  X_test_ithday.append(df.iloc[start_index-15:start_index,4:].values)
  
  

  y_test_ithday.append(df.iloc[start_index,4])
  

 

for i in dates:
  X_test_ithday=list()
  y_test_ithday=list()
  
  df_scaled.groupby(["Symbols"]).apply(prepare_test_daily, i)

  X_test_ithday=np.array(X_test_ithday)
  y_test_ithday=np.array(y_test_ithday)

  if(X_test_ithday.shape == (0,)):
    continue

  X_testdaily.append(X_test_ithday)
  y_testdaily.append(y_test_ithday)
  ithday_date.append(i)



In [107]:
len(X_testdaily)

23

In [109]:
precisions=[]
accuracies=[]
for i in range(13):
  ithday_predictions=saved_model.predict_classes(X_testdaily[i])
  ithday_predictions=ithday_predictions.reshape(1,-1)[0]
  precision = precision_score(y_testdaily[i],ithday_predictions)
  # print(precision , i)
  accuracy=accuracy_score(y_testdaily[i],ithday_predictions)
  precisions.append(precision)
  accuracies.append(accuracy)
accuracies=np.array(accuracies)
precisions=np.array(precisions)

print("Average Per Day Accuracy for the month of January 2020 - ",accuracies.mean())
print("Standard Deviation of accuracies for the month of January 2020 -",accuracies.std())
print("\nAverage Precision per Day for the month of January 2020 - ",precisions.mean())
print("Standard Deviations of precisions for the month of January 2020 -",precisions.std())

Average Per Day Accuracy for the month of January 2020 -  0.7722399594903349
Standard Deviation of accuracies for the month of January 2020 - 0.11226027329855881

Average Precision per Day for the month of January 2020 -  0.5911671661671661
Standard Deviations of precisions for the month of January 2020 - 0.2697101902961413


In [38]:
jan_2nd_predictions = saved_model.predict_classes(X_test_2ndjan)
jan_2nd_predictions=jan_2nd_predictions.reshape(1,-1)[0]

precision = precision_score(y_test_2ndjan, jan_2nd_predictions)
accuracy=accuracy_score(y_test_2ndjan,jan_2nd_predictions)

print("Train Precision is ",precision)
print("Train Accuracy is ",accuracy)

pd.crosstab(y_test_2ndjan, jan_2nd_predictions, rownames=['True'], colnames=['Predicted'], margins=True)



Train Precision is  0.75
Train Accuracy is  0.9637618636755824


Predicted,0,1,All
True,,,
0.0,1111,2,1113
1.0,40,6,46
All,1151,8,1159


In [0]:
### IGNore code below

In [0]:
### Ignore COde below

In [0]:
### This is another model I tried but is not good

In [0]:
from keras.models import load_model
saved_model = load_model('best_model.h5')

In [0]:
from sklearn.metrics import precision_score,accuracy_score,classification_report
y_predicted=saved_model.predict_classes(X_train)
y_predicted=y_predicted.reshape(1,-1)[0]
precision = precision_score(y_train, y_predicted)
accuracy=accuracy_score(y_train,y_predicted)
print("Train Precision is ",precision)
print("Train Accuracy is ",accuracy)

Train Precision is  0.68
Train Accuracy is  0.8362445247941781


In [0]:
pd.crosstab(y_train, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0.0,218178,288,218466
1.0,42556,612,43168
All,260734,900,261634


In [0]:
y_predicted=saved_model.predict_classes(X_test)
y_predicted=y_predicted.reshape(1,-1)[0]
precision = precision_score(y_test, y_predicted)
accuracy=accuracy_score(y_test,y_predicted)
print("Test Precision is ",precision)
print("Test Accuracy is ",accuracy)

Test Precision is  0.8181818181818182
Test Accuracy is  0.8140758553367777


In [0]:
pd.crosstab(y_test, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0.0,34147,10,34157
1.0,7799,45,7844
All,41946,55,42001


In [0]:
model=Sequential()

model.add(LSTM(units=50,input_shape=(X_train.shape[1],45)))
model.add(Dropout(0.2))

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(128,activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(units=1,activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

class_weight={0:0.40,
               1:0.60}

mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['binary_accuracy'])

In [0]:
model.fit(X_train, y_train, batch_size=500, nb_epoch=200,validation_split=0.33,callbacks=[es,mc],class_weight=class_weight)

Train on 175294 samples, validate on 86340 samples
Epoch 1/200
   500/175294 [..............................] - ETA: 17s - loss: 0.2499 - binary_accuracy: 0.8120

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


175294/175294 [==============================] - 12s 68us/step - loss: 0.2291 - binary_accuracy: 0.8352 - val_loss: 0.2275 - val_binary_accuracy: 0.8344

Epoch 00001: val_loss improved from inf to 0.22755, saving model to best_model.h5
Epoch 2/200
175294/175294 [==============================] - 12s 69us/step - loss: 0.2277 - binary_accuracy: 0.8349 - val_loss: 0.2272 - val_binary_accuracy: 0.8348

Epoch 00002: val_loss improved from 0.22755 to 0.22719, saving model to best_model.h5
Epoch 3/200
175294/175294 [==============================] - 12s 68us/step - loss: 0.2271 - binary_accuracy: 0.8342 - val_loss: 0.2262 - val_binary_accuracy: 0.8348

Epoch 00003: val_loss improved from 0.22719 to 0.22621, saving model to best_model.h5
Epoch 4/200
175294/175294 [==============================] - 12s 66us/step - loss: 0.2269 - binary_accuracy: 0.8348 - val_loss: 0.2262 - val_binary_accuracy: 0.8350

Epoch 00004: val_loss improved from 0.22621 to 0.22617, saving model to best_model.h5
Epoch 5/

In [0]:
from keras.models import load_model
saved_model = load_model('best_model.h5')

In [0]:
from sklearn.metrics import precision_score,accuracy_score,classification_report
y_predicted=saved_model.predict_classes(X_train)
y_predicted=y_predicted.reshape(1,-1)[0]
precision = precision_score(y_train, y_predicted)
accuracy=accuracy_score(y_train,y_predicted)
print("Train Precision is ",precision)
print("Train Accuracy is ",accuracy)

Train Precision is  0.5363372093023255
Train Accuracy is  0.8357705802762638


In [0]:
pd.crosstab(y_train, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0.0,217190,1276,218466
1.0,41692,1476,43168
All,258882,2752,261634


In [0]:
y_predicted=saved_model.predict_classes(X_test)
y_predicted=y_predicted.reshape(1,-1)[0]
precision = precision_score(y_test, y_predicted)
accuracy=accuracy_score(y_test,y_predicted)
print("Test Precision is ",precision)
print("Test Accuracy is ",accuracy)

Test Precision is  0.5664739884393064
Test Accuracy is  0.8137901478536226


In [0]:
pd.crosstab(y_test, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0.0,34082,75,34157
1.0,7746,98,7844
All,41828,173,42001
